# Load News data from NewsAPI.org


## First: Install python NewsAPI.org package

Note: both pip and pip3 here. Please use the corresponding one


In [33]:
%pip install newsapi-python
%pip3 install newsapi-python


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%pip3` not found.


## Configure the NewsApiClient to connect to the API

Thanks to whoever donated their key here; appreciated.


In [34]:
from newsapi import NewsApiClient
#my api KEY: 00b3f189be5749b8b82314b9d9d5c5ef
api = NewsApiClient(api_key='00b3f189be5749b8b82314b9d9d5c5ef')

## Use pandas to store data

sourceList --> stores the journal name (e.g., The Verge) from the source
</br>
timeList --> stores article publish date (UTC ISO 8061 format "YYYY-MM-DDTHH-MM:SSZ")
</br>
desList --> stores summary/description of the article
</br>
</br>
NOTE: for raw data go here: https://newsapi.org/v2/everything?q=tesla&apiKey=00b3f189be5749b8b82314b9d9d5c5ef
</br>
Also, note that current query is for the string "tesla" only.


In [35]:
# from numpy import source -- is this really needed here?
import pandas as pd
des_list = []
source_list = []
time_list = []
news_data = (api.get_everything(q='tesla'))
for article in news_data['articles']:
  source_list.append(article['source']['name'])
  time_list.append(article['publishedAt'])
  des_list.append(article['description'])


## Run Sentiment Analysis on the data

NOTE: if on MacOS and getting "SSL CERTIFICATE_VERIFY_FAILED" error, then follow these instructions to install the certificates: https://stackoverflow.com/questions/40684543/how-to-make-python-use-ca-certificates-from-mac-os-truststore


In [36]:
# import the needed modules
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
# Credit for vader module:
'''Hutto, C.J. & Gilbert, E.E. (2014). 
VADER: A Parsimonious Rule-based Model for Sentiment Analysis of Social Media Text. 
Eighth International Conference on Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014.'''

# Function to analyze each text [description]
sentiment_list = []
def nltk_sentiment(sentence):
  # NLTK analyzer provides 4 datapoints in a dictionary --> compound datapoint is used; 
  # Range is -1 to 1 where -1 being most negative and 1 being most positive
  analyzer = SentimentIntensityAnalyzer()
  return analyzer.polarity_scores(sentence)["compound"]

# call function for all data in dataset and store --> sentiment_list
for i in des_list:
  sentiment_list.append(nltk_sentiment(i))

# Store in a Dataframe object; # Structure: 
# "Sources" [e.g., The Verge], "Description" [Summary text], "Publishing" [Date of publish], "Sentiments" [Sentiment on scale of -1 to 1]
news_df = pd.DataFrame(list(zip(source_list, des_list, time_list, sentiment_list)), columns=['Sources','Description','Publishing','Sentiments'])
display(news_df)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/saad/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,Sources,Description,Publishing,Sentiments
0,The Verge,Tesla reported its Q3 earnings for 2023 in whi...,2023-10-18T20:44:47Z,0.4404
1,The Verge,Tesla will deliver the first units of its elec...,2023-10-18T20:58:48Z,-0.2960
2,The Verge,Hyundai will feature Tesla’s EV charging port ...,2023-10-05T12:05:00Z,0.1779
3,Engadget,Huawei\r\n is moving deeper into the electric ...,2023-09-25T20:09:18Z,0.4019
4,Engadget,NASA's pioneering OSIRIS-REx mission has succe...,2023-09-25T19:21:32Z,0.4939
...,...,...,...,...
95,heise online,"Bei Tesla gibt es zurzeit vier große Projekte,...",2023-09-25T05:24:00Z,-0.5994
96,Uncrate.com,Safari all the cars. From classic and factory-...,2023-10-17T21:55:46Z,0.0790
97,Golem.de,Tesla greift zu radikalen Maßnahmen: Massive P...,2023-10-07T12:10:02Z,-0.5994
98,Ars Technica,Nissan has been able to satisfy the IRS's dome...,2023-10-16T16:16:28Z,0.4588
